## Prueba Final

In [ ]:
!pip install -q google-generativeai

In [1]:
import google.generativeai as genai
from google.colab import userdata

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [2]:
API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)

In [3]:
model = genai.GenerativeModel("gemini-2.5-flash")

In [4]:
class MotorIA:
  def __init__(self, model):
    self.model = model

  def generar(self, prompt:str) -> str:
    response = self.model.generate_content(prompt)
    return response.text.strip()

In [42]:
def construir_prompt(actividad: dict) -> str:
  prompt_actividad = f"""
Eres un experto en levantamiento de cargas laborales.

Analiza la siguiente actividad y entrega recomendaciones
claras y prácticas, teniendo en cuenta el cargo de la persona y la vicepresidencia en la que trabaja.

Actividad:
- Vicepresidencia: {actividad["vicepresidencia"]}
- Cargo: {actividad["cargo"]}
- Unidad de medida: {actividad["unidad_medida"]}
- Descripción: {actividad["descripcion"]}
- Frecuencia: {actividad["frecuencia"]}
- Volumen: {actividad["volumen"]}
- Tiempo real (min): {actividad["tiempo_real_min"]}
- Proceso del área: {actividad["proceso_area"]}
- Autonomía: {actividad["autonomia"]}%
- Etapa PHVA: {actividad["phva"]}
- Observaciones: {actividad["observaciones"]}

Entrega de forma resumida y concisa:
- Riesgos
- Oportunidades de mejora
- Recomendaciones concretas
"""
  return prompt_actividad

In [43]:
import time

def medir_tiempo_real():
  input("\nPresiona ENTER para INICIAR la actividad...")
  inicio = time.time()

  input("Presiona ENTER para FINALIZAR la actividad...")
  fin = time.time()

  segundos = fin-inicio
  minutos = round(segundos/60, 2)

  print(f"\nTiempo registrado: {minutos} minutos.")

  return minutos

In [44]:
def formulario_actividad():
    print("\n📝 Registro de actividad\n")

    actividad = {}

    actividad["vicepresidencia"] = input(
        "1️⃣ ¿Cuál es su Vicepresidencia?:\n> "
    )

    actividad["cargo"] = input(
        "1️⃣ ¿Cuál es su cargo?:\n> "
    )

    actividad["descripcion"] = input(
        "1️⃣ Describe la actividad que realizas:\n> "
    )

    actividad["unidad_medida"] = input(
        "2️⃣ Unidad de medida (Correo, Reunión, Documento, etc.):\n> "
    )

    actividad["frecuencia"] = input(
        "3️⃣ Frecuencia (diaria / semanal / mensual / anual):\n> "
    )

    actividad["volumen"] = int(input(
        "4️⃣ Volumen según la frecuencia:\n> "
    ))

    #actividad["tiempo_real_min"] = medir_tiempo_real()
    actividad["tiempo_real_min"] = int(input(
        "5️⃣ Tiempo real (minutos) en esta actividad:\n> "
    ))

    actividad["proceso_area"] = input(
        "6️⃣ Proceso del área al que pertenece:\n> "
    )

    actividad["autonomia"] = int(input(
        "7️⃣ Autonomía (%) en esta actividad:\n> "
    ))

    actividad["phva"] = input(
        "8️⃣ Etapa PHVA (Planear / Hacer / Verificar / Actuar):\n> "
    )

    actividad["observaciones"] = input(
        "9️⃣ Observaciones (opcional):\n> "
    )

    return actividad


In [45]:
estado = {
    "rol": None,
    "actividades": [],
    "actividad_actual": None
}

In [46]:
def seleccionar_rol():
  print("Seleccione su rol:")
  print("1-Registro de Actividades")
  print("2-Analista de Cargas")

  opcion = input("Opcion: ")

  if opcion == "1":
    return "registro"
  elif opcion == "2":
    return "analista"
  else:
    print("Opcion no valida")
    return seleccionar_rol()

In [47]:
def factor_frecuencia(frecuencia):
  frecuencia = frecuencia.lower()

  if frecuencia == "diaria":
    return 1
  if frecuencia == "semanal":
    return 1/5
  if frecuencia == "mensual":
    return 1/20
  if frecuencia == "anual":
    return 1/240

  return 0

In [48]:
def calcular_metricas_actividad(actividad):
  minutos_base = actividad["tiempo_real_min"]
  volumen = actividad["volumen"]
  factor = factor_frecuencia(actividad["frecuencia"])
  autonomia = actividad["autonomia"] / 100

  minutos_diarios = minutos_base*volumen*factor*(1-autonomia)

  return {
      "minutos_diarios": round(minutos_diarios,2),
      "horas_diarias": round(minutos_diarios/60,2),
  }

In [49]:
def calcular_resumen_global(actividades):
  total_minutos = 0

  for act in actividades:
    metricas = calcular_metricas_actividad(act)
    act["metricas"] = metricas
    total_minutos += metricas["minutos_diarios"]

  horas_totales = total_minutos/60
  horas_efectivas = 7.3

  personas_requeridas = round(horas_totales/horas_efectivas, 2)
  carga_total = round((horas_totales/8)*100, 2)

  return {
      "horas_diarias_total": round(horas_totales, 2),
      "personas_requeridas": personas_requeridas,
      "porcentaje_carga_total": carga_total,
      "minutos_diarios_total": round(total_minutos, 2)
  }

In [50]:
motor = MotorIA(model)

actividades = []

In [51]:
estado["rol"] = seleccionar_rol()

Seleccione su rol:
1-Registro de Actividades
2-Analista de Cargas
Opcion: 1


In [52]:
if estado["rol"] == "registro":
  print("\n📋 REGISTRO DE ACTIVIDADES")
  print("=" * 40)

  while True:
      actividad = formulario_actividad()

      prompt = construir_prompt(actividad)
      recomendaciones = motor.generar(prompt)

      actividad["recomendaciones"] = recomendaciones
      estado["actividades"].append(actividad)

      actividades.append(actividad)

      print("\n🤖 Recomendaciones para esta actividad:\n")
      print(recomendaciones)

      continuar = input("\n¿Deseas registrar otra actividad? (s/n): ").lower()
      if continuar != "s":
          break

elif estado["rol"] == "analista":
  print("\n🟥 MODO ANALISTA DE CARGAS")
  print("=" * 40)

  if not actividades:
      print("❌ No hay actividades registradas.")
  else:
      resumen = calcular_resumen_global(actividades)

      print("\n📊 RESUMEN GLOBAL")
      print("=" * 40)
      for k, v in resumen.items():
          print(f"{k}: {v}")

      prompt_resumen = f"""
Eres un experto en análisis de cargas laborales.

Analiza el siguiente resumen y entrega:
- Diagnóstico general
- Riesgos de sobrecarga
- Tiempo muerto potencial
- Recomendación de dotación

Resumen:
{resumen}
"""
      analisis_ia = motor.generar(prompt_resumen)

      print("\n🤖 Análisis IA del resumen:\n")
      print(analisis_ia)



📋 REGISTRO DE ACTIVIDADES

📝 Registro de actividad

1️⃣ ¿Cuál es su Vicepresidencia?:
> Vp de Talento y Administrativa
1️⃣ ¿Cuál es su cargo?:
> Gerente
1️⃣ Describe la actividad que realizas:
> Liderar  el diseño y actualización del roadmap de  Delivery en Negocio
2️⃣ Unidad de medida (Correo, Reunión, Documento, etc.):
> Comite primario
3️⃣ Frecuencia (diaria / semanal / mensual / anual):
> semanal
4️⃣ Volumen según la frecuencia:
> 1
5️⃣ Tiempo real (minutos) en esta actividad:
> 60
6️⃣ Proceso del área al que pertenece:
> Desarrollo institucional de nuevas formas de trabajo y agilidad
7️⃣ Autonomía (%) en esta actividad:
> 100
8️⃣ Etapa PHVA (Planear / Hacer / Verificar / Actuar):
> Planear
9️⃣ Observaciones (opcional):
> Sesion de alineación de prioridades con el equipo de leads de agilidad 

🤖 Recomendaciones para esta actividad:

¡Excelente análisis de actividad! Como experto en levantamiento de cargas laborales, valoro la precisión de los datos.

Aquí mi análisis resumido y co

In [53]:
def participacion_por_proceso(actividades):
  total_minutos = sum(act["metricas"]["minutos_diarios"] for act in actividades)

  procesos = {}

  for act in actividades:
    proceso = act["proceso_area"]
    minutos = act["metricas"]["minutos_diarios"]

    procesos.setdefault(proceso, 0)
    procesos[proceso] += minutos

  for proceso in procesos:
    procesos[proceso] = {
        "minutos_diarios": round(procesos[proceso], 2),
        "porcentaje": round((procesos[proceso]/total_minutos)*100, 2)
    }

  return procesos

In [54]:
HORAS_EFECTIVAS = 7.3

def calcular_carga_final(resumen):
  horas_totales = resumen["horas_diarias_total"]
  porcentaje_carga = round((horas_totales/HORAS_EFECTIVAS)*100, 2)
  tiempo_muerto = round(max(0, horas_totales - HORAS_EFECTIVAS), 2)

  return {
      "porcentaje_carga_final": porcentaje_carga,
      "tiempo_muerto_horas": tiempo_muerto
  }

In [55]:
def calcular_dotacion(horas_totales):
  personas = round(horas_totales/HORAS_EFECTIVAS, 2)
  return personas

In [56]:
def resumen_ejecutivo(actividades):
  resumen_base = calcular_resumen_global(actividades)
  carga_final = calcular_carga_final(resumen_base)
  procesos = participacion_por_proceso(actividades)

  return {
      **resumen_base,
      **carga_final,
      "participacion_por_proceso": procesos,
      "dotacion_recomendada": calcular_dotacion(resumen_base["horas_diarias_total"])
  }

In [58]:
resumen_final = resumen_ejecutivo(actividades)

print("\n📊 RESUMEN EJECUTIVO FINAL")
print("=" * 50)

for k, v in resumen_final.items():
    if k != "participacion_por_proceso": # Corrected key
        print(f"{k}: {v}")

print("\n📌 PARTICIPACIÓN POR PROCESO")
for proc, datos in resumen_final["participacion_por_proceso"].items(): # Corrected key
    print(f"- {proc}: {datos['porcentaje']}% ({datos['minutos_diarios']} min)")


📊 RESUMEN EJECUTIVO FINAL
horas_diarias_total: 0.15
personas_requeridas: 0.02
porcentaje_carga_total: 1.9
minutos_diarios_total: 9.12
porcentaje_carga_final: 2.05
tiempo_muerto_horas: 0
dotacion_recomendada: 0.02

📌 PARTICIPACIÓN POR PROCESO
- Desarrollo institucional de nuevas formas de trabajo y agilidad: 0.0% (0.0 min)
- Consolidación del equipo de Cultura y Transformación: 100.0% (9.12 min)


In [59]:
prompt_dotacion = f"""
Eres un consultor experto en productividad y dotación.

Analiza este resumen de carga laboral y entrega:
- Diagnóstico claro
- Riesgos principales
- Recomendación de dotación
- Alertas de tiempo muerto o sobrecarga

Resumen:
{resumen_final}
"""

analisis_ia = motor.generar(prompt_dotacion)

print("\n🤖 ANÁLISIS IA FINAL\n")
print(analisis_ia)


🤖 ANÁLISIS IA FINAL

¡Excelente! Como su consultor experto en productividad y dotación, he analizado el resumen de carga laboral proporcionado. La información es sumamente reveladora, aunque de una manera muy particular.

---

### Análisis de Carga Laboral y Dotación

**Diagnóstico Claro:**

El diagnóstico principal es que **la carga laboral reportada es extremadamente baja, casi insignificante, para justificar cualquier tipo de dotación dedicada o rol principal.**

*   **Carga Mínima:** El tiempo total de trabajo diario es de solo **9.12 minutos (0.15 horas)**, lo que representa apenas un **2.05% de la carga de un día laboral estándar** (asumiendo 8 horas).
*   **Concentración Absoluta:** El 100% de este tiempo (los 9.12 minutos) está dedicado a un único proceso: "Consolidación del equipo de Cultura y Transformación". El proceso de "Desarrollo institucional de nuevas formas de trabajo y agilidad" registra 0 minutos de actividad.
*   **Dotación Recomendada Impracticable:** La recomend

In [62]:
from datetime import datetime
import json

In [61]:
def construir_registro_historico(rol, actividades, resumen_final, analisis_ia):
  return {
      "timestamp": datetime.now().isoformat(),
      "rol": rol,
      "actividades": actividades,
      "resumen_final": resumen_final,
      "analisis_ia": analisis_ia
  }

In [63]:
def guardar_historico(registro, nombre_archivo=None):
  if not nombre_archivo:
    fecha = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"historico_{fecha}.json"

  with open(nombre_archivo, "w", encoding="utf-8") as f:
    json.dump(registro, f, indent=2, ensure_ascii=False)

  print(f"\nHistorico guardado en: {nombre_archivo}")

In [64]:
registro = construir_registro_historico(
    rol=estado["rol"],
    actividades=actividades,
    resumen_final=resumen_final,
    analisis_ia=analisis_ia
)

guardar_historico(registro)


Historico guardado en: historico_20260115_154146.json


In [65]:
with open("historico_20260115_154146.json", 'r', encoding='utf-8') as f:
  data = json.load(f)

# Step 3: Pretty-print JSON with indentation
print("\n--- JSON Content ---")
print(json.dumps(data, indent=4, ensure_ascii=False))


--- JSON Content ---
{
    "timestamp": "2026-01-15T15:41:46.333358",
    "rol": "registro",
    "actividades": [
        {
            "vicepresidencia": "Vp de Talento y Administrativa",
            "cargo": "Gerente",
            "descripcion": "Liderar  el diseño y actualización del roadmap de  Delivery en Negocio",
            "unidad_medida": "Comite primario",
            "frecuencia": "semanal",
            "volumen": 1,
            "tiempo_real_min": 60,
            "proceso_area": "Desarrollo institucional de nuevas formas de trabajo y agilidad",
            "autonomia": 100,
            "phva": "Planear",
            "observaciones": "Sesion de alineación de prioridades con el equipo de leads de agilidad ",
            "recomendaciones": "¡Excelente análisis de actividad! Como experto en levantamiento de cargas laborales, valoro la precisión de los datos.\n\nAquí mi análisis resumido y conciso:\n\n**Contexto Clave:** El Gerente de la VP de Talento y Administrativa lidera un